<a href="https://colab.research.google.com/github/prathamgarg911/Pytorch/blob/main/ResNet_18_from_scratch_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision.transforms import v2
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
batch_dict = {}
test_dict = unpickle('/content/drive/MyDrive/computer-vision-pytorch/cifar-10-batches-py/test_batch')
for i in range(5):
    batch_dict[f'batch_{i+1}'] = unpickle("/content/drive/MyDrive/computer-vision-pytorch/cifar-10-batches-py/data_batch_%d"%(i + 1))

In [ ]:
norm_train_batch = {}
ls = [x for x in batch_dict.keys()]

In [ ]:
for name in ls:
    temp = torch.tensor(batch_dict[name][b'data'],dtype = torch.float32)
    temp = temp/255.0
    temp = temp.view(temp.shape[0],3,32,32)
    normalize = transforms.Compose([
    transforms.Normalize(mean=[torch.mean(temp[:,0,:,:]).item(), torch.mean(temp[:,1,:,:]).item(), torch.mean(temp[:,2,:,:]).item()], std=[torch.std(temp[:,0,:,:]).item(), torch.std(temp[:,1,:,:]).item(), torch.std(temp[:,2,:,:]).item()])  # Normalize and scale
    ])
    temp = normalize(temp)
    norm_train_batch[name] = temp


test_images = torch.tensor(test_dict[b'data'],dtype = torch.float32)
test_labels = torch.tensor(test_dict[b'labels'])
test_images = test_images.view(test_images.shape[0],3,32,32)
norm_test_images = test_images/255.0
norm_test_images = norm_test_images.view(norm_test_images.shape[0],3,32,32)

normalizer = transforms.Compose([
    transforms.Normalize(mean=[torch.mean(norm_test_images[:,0,:,:]).item(), torch.mean(norm_test_images[:,1,:,:]).item(), torch.mean(norm_test_images[:,2,:,:]).item()], std=[torch.std(norm_test_images[:,0,:,:]).item(), torch.std(norm_test_images[:,1,:,:]).item(), torch.std(norm_test_images[:,2,:,:]).item()])
])

norm_test_images = normalizer(norm_test_images)

training_image = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'data']),torch.tensor(batch_dict['batch_2'][b'data']),torch.tensor(batch_dict['batch_3'][b'data']),torch.tensor(batch_dict['batch_4'][b'data']),torch.tensor(batch_dict['batch_5'][b'data'])),0),dtype = torch.float32)
training_labels = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'labels']),torch.tensor(batch_dict['batch_2'][b'labels']),torch.tensor(batch_dict['batch_3'][b'labels']),torch.tensor(batch_dict['batch_4'][b'labels']),torch.tensor(batch_dict['batch_5'][b'labels']))))


norm_training_image = torch.tensor(torch.cat((torch.tensor(norm_train_batch['batch_1']),torch.tensor(norm_train_batch['batch_2']),torch.tensor(norm_train_batch['batch_3']),torch.tensor(norm_train_batch['batch_4']),torch.tensor(norm_train_batch['batch_5'])),0),dtype = torch.float32)

<ipython-input-10-b47502e7a676>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  training_image = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'data']),torch.tensor(batch_dict['batch_2'][b'data']),torch.tensor(batch_dict['batch_3'][b'data']),torch.tensor(batch_dict['batch_4'][b'data']),torch.tensor(batch_dict['batch_5'][b'data'])),0),dtype = torch.float32)
<ipython-input-10-b47502e7a676>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  training_labels = torch.tensor(torch.cat((torch.tensor(batch_dict['batch_1'][b'labels']),torch.tensor(batch_dict['batch_2'][b'labels']),torch.tensor(batch_dict['batch_3'][b'labels']),torch.tensor(batch_dict['batch_4'][b'labels']),torch.ten

In [ ]:
training_image = training_image.view(50000,3,32,32)

In [ ]:
test_images.shape

torch.Size([10000, 3, 32, 32])

In [ ]:
training_labels.shape

torch.Size([50000])

In [ ]:
test_labels.shape

torch.Size([10000])

In [68]:
class Resnet(nn.Module):

  def __init__ (self):

    super(Resnet,self).__init__()

    self.pad = nn.ZeroPad2d(2)

    self.Conv0 = nn.Conv2d(3,64,3,padding=1)
    self.Conv1 = nn.Conv2d(64,64,3,padding=1)
    self.Conv2 = nn.Conv2d(64,128,3,padding=1)
    self.Conv3 = nn.Conv2d(128,128,3,padding=1)
    self.Conv4 = nn.Conv2d(128,256,3,padding=1)
    self.Conv5 = nn.Conv2d(256,256,3,padding=1)
    self.Conv6 = nn.Conv2d(256,512,3,padding=1)
    self.Conv7 = nn.Conv2d(512,512,3,padding=1)
    self.maxpool = nn.MaxPool2d(2)

    self.bnorm1 = nn.BatchNorm2d(64)
    self.bnorm2 = nn.BatchNorm2d(128)
    self.bnorm3 = nn.BatchNorm2d(256)
    self.bnorm4 = nn.BatchNorm2d(512)

    self.avgpool = nn.AvgPool2d(2)

    self.Linear1 = nn.Linear(512*2*2,512)
    self.Linear2 = nn.Linear(512,10)

    self.softmax = nn.Softmax(1)


  def forward(self,x) :

     x = self.Conv0(x)

     shortcut = x

     x = F.relu(self.Conv1(x))
     x = F.relu(self.bnorm1(self.Conv1(x)))
     x = x + shortcut

     shortcut = x

     x = F.relu(self.Conv1(x))
     x = F.relu(self.bnorm1(self.Conv1(x)))
     x = x + shortcut

     shortcut = F.relu(self.bnorm2(self.Conv2(x)))
     shortcut = self.maxpool(shortcut)

     x = F.relu(self.bnorm2(self.Conv2(x)))
     x = F.relu(self.bnorm2(self.Conv3(x)))
     x = self.maxpool(x)
     x = x + shortcut

     shortcut = x

     x = F.relu(self.bnorm2(self.Conv3(x)))
     x = F.relu(self.bnorm2(self.Conv3(x)))
     x = x + shortcut

     shortcut = F.relu(self.bnorm3(self.Conv4(x)))
     shortcut = self.maxpool(shortcut)

     x = F.relu(self.bnorm3(self.Conv4(x)))
     x = F.relu(self.bnorm3(self.Conv5(x)))
     x = self.maxpool(x)
     x = x + shortcut

     shortcut = x

     x = F.relu(self.bnorm3(self.Conv5(x)))
     x = F.relu(self.bnorm3(self.Conv5(x)))
     x = x + shortcut

     shortcut = F.relu(self.bnorm4(self.Conv6(x)))
     shortcut = self.maxpool(shortcut)

     x = F.relu(self.bnorm4(self.Conv6(x)))
     x = F.relu(self.bnorm4(self.Conv7(x)))
     x = self.maxpool(x)
     x = x + shortcut

     shortcut = x

     x = F.relu(self.bnorm4(self.Conv7(x)))
     x = F.relu(self.bnorm4(self.Conv7(x)))
     x = x + shortcut


     x = self.avgpool(x)

     x = x.view(x.shape[0],512*2*2)
     x = F.relu(self.Linear1(x))

     x = self.Linear2(x)
    #  x = self.softmax(x)


     return x








In [69]:
batch_size = 16
train_dataset = TensorDataset(training_image,training_labels)
test_dataset = TensorDataset(test_images,test_labels)
train_loader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
test_loader = DataLoader(test_dataset,shuffle =True,batch_size=batch_size)

In [70]:
device = 'cuda'
model = Resnet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr = 0.001)


In [ ]:
training_loss = 0.0
training_accuracy = 0

for epoch in range(20):
    for i,(images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        loss = criterion(output,labels)

        # training_loss += loss.item()
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        _, predicted = torch.max(output.data, 1)

        training_accuracy += (predicted == labels).sum().item()
        if ((i+1)%100 == 0):
            print(f'epoch:{epoch + 1},loss:{loss.item():.4f}')
            # wandb.log({
            #     "Training_Loss":training_loss/100,
            #     "Training_accuracy":training_accuracy/100,
            # })
            # training_loss = 0.0
            # training_accuracy = 0
    print(f'accuracy:{training_accuracy*100/50000}')
    training_accuracy = 0

epoch:1,loss:2.0920
epoch:1,loss:2.2537
epoch:1,loss:2.2597
epoch:1,loss:1.6909
epoch:1,loss:1.9126
epoch:1,loss:1.8822
epoch:1,loss:1.6752
epoch:1,loss:1.7822
epoch:1,loss:1.5613
epoch:1,loss:1.1913
epoch:1,loss:1.4774
epoch:1,loss:1.3928
epoch:1,loss:1.2924
epoch:1,loss:1.5856
epoch:1,loss:1.4603
epoch:1,loss:1.4893
epoch:1,loss:1.9328
epoch:1,loss:1.3559
epoch:1,loss:1.3927
epoch:1,loss:1.3885
epoch:1,loss:1.2092
epoch:1,loss:1.1976
epoch:1,loss:1.2256
epoch:1,loss:1.5887
epoch:1,loss:0.8023
epoch:1,loss:1.6595
epoch:1,loss:1.1923
epoch:1,loss:1.1918
epoch:1,loss:1.6131
epoch:1,loss:1.1398
epoch:1,loss:0.6853
accuracy:42.716
epoch:2,loss:1.0190
epoch:2,loss:0.5948
epoch:2,loss:0.8578
epoch:2,loss:1.0930
epoch:2,loss:0.9092
epoch:2,loss:0.7443
epoch:2,loss:1.5432
epoch:2,loss:0.7981
epoch:2,loss:0.6783
epoch:2,loss:1.1956
epoch:2,loss:1.3413
epoch:2,loss:0.8752
epoch:2,loss:0.9483
epoch:2,loss:0.6848
epoch:2,loss:0.8754
epoch:2,loss:0.9827
epoch:2,loss:1.0083
epoch:2,loss:0.7064
epoc

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    test_loss = 0.0
    test_accuracy = 0
    for i,(images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs.data,labels)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        test_accuracy = n_correct
        test_loss = loss.item()
        if((i+1)%50 == 0):
            # wandb.log({"Test_loss":test_loss/100,
            #            "Test_accuracy":test_accuracy/100})
            test_accuracy = 0
            test_loss = 0
    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images:{acc}%')

Accuracy of the network on the 10000 test images:77.65%


In [ ]:
!nvidia-smi